In [1]:
# Import deepos
import deepops as dp

# Give your tensor a cool name.

In [2]:
# make Tensors
a = dp.Tensor([1.0]*100000, name = "leo")
b = dp.Tensor([2.0]*100000, name = "brad pitt")

In [17]:
print(a.name)
print(b.name)

leo
brad pitt


# Forward Operations.

In [4]:
# Tensor Operations on CPU
print("Add:", a+b, "Mul:", a*b)

Add: dp.Tensor shape: (100000,), numpy: ([3. 3. 3. ... 3. 3. 3.], dtype=float32), cuda device: cpu  Mul: dp.Tensor shape: (100000,), numpy: ([2. 2. 2. ... 2. 2. 2.], dtype=float32), cuda device: cpu 


In [5]:
# Default Device is cpu
a

dp.Tensor shape: (100000,), numpy: ([1. 1. 1. ... 1. 1. 1.], dtype=float32), cuda device: cpu 

In [6]:
# throw these Tensors in our precious GPU
a.device('gpu') # i.e gpu:0
b.device('gpu')

dp.Tensor shape: (100000,), numpy: (<pycuda._driver.DeviceAllocation object at 0x7f855c6655d0>, dtype=float32), cuda device: gpu 

In [7]:
# add them on GPU
addition = a.add(b)

In [8]:
# print the Addition
addition

dp.Tensor shape: (100000,), numpy: (<pycuda._driver.DeviceAllocation object at 0x7f85292b7120>, dtype=float32), cuda device: gpu 

In [9]:
# adds beautifully with itself!
a.add(a)

dp.Tensor shape: (100000,), numpy: (<pycuda._driver.DeviceAllocation object at 0x7f85292b7e40>, dtype=float32), cuda device: gpu 

In [10]:
print(a + a)

dp.Tensor shape: (100000,), numpy: (<pycuda._driver.DeviceAllocation object at 0x7f8529306ad0>, dtype=float32), cuda device: gpu 


In [11]:
print(a * b)

dp.Tensor shape: (100000,), numpy: (<pycuda._driver.DeviceAllocation object at 0x7f85292b7df0>, dtype=float32), cuda device: gpu 


In [12]:
repr(a)

'dp.Tensor shape: (100000,), numpy: (<pycuda._driver.DeviceAllocation object at 0x7f85292b7170>, dtype=float32), cuda device: gpu '

# BackPropogration is real now.

In [13]:
Tensor = dp.Tensor
a1 = Tensor([1.0, 3.0, 1.0])
b1 = Tensor([7.0, 3.0, 5.0])

a2 = Tensor([4.0, 3.0, 1.0])
a3 = Tensor([3.0, 3.0, 1.0])
a4 = Tensor([7.0, 1.0, 6.0])
b2 = Tensor([1.0, 21.0, 12.0])

c = a1 * b1 + a3
d = a2 * b2 + a4
out = c * d

In [14]:
# calculate gradients in style!!
out.backward()

1

In [15]:
# output gradients
out.grad

1

In [16]:
# Gradients
_ = [print(f"Gradients for {tensor} tensor is : {tensor.grad}") for tensor in [a1,a2,a3,a4,b1,b2,c,d]]

Gradients for dp.Tensor shape: (3,), numpy: ([1. 3. 1.], dtype=float32), cuda device: cpu  tensor is : [ 77. 192.  90.]
Gradients for dp.Tensor shape: (3,), numpy: ([4. 3. 1.], dtype=float32), cuda device: cpu  tensor is : [ 10. 252.  72.]
Gradients for dp.Tensor shape: (3,), numpy: ([3. 3. 1.], dtype=float32), cuda device: cpu  tensor is : [11. 64. 18.]
Gradients for dp.Tensor shape: (3,), numpy: ([7. 1. 6.], dtype=float32), cuda device: cpu  tensor is : [10. 12.  6.]
Gradients for dp.Tensor shape: (3,), numpy: ([7. 3. 5.], dtype=float32), cuda device: cpu  tensor is : [ 11. 192.  18.]
Gradients for dp.Tensor shape: (3,), numpy: ([ 1. 21. 12.], dtype=float32), cuda device: cpu  tensor is : [40. 36.  6.]
Gradients for dp.Tensor shape: (3,), numpy: ([10. 12.  6.], dtype=float32), cuda device: cpu  tensor is : [11. 64. 18.]
Gradients for dp.Tensor shape: (3,), numpy: ([11. 64. 18.], dtype=float32), cuda device: cpu  tensor is : [10. 12.  6.]
